In [26]:
import pickle
import glob
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

disease = "crc"

files = glob.glob('results/'+disease+'/'+disease+'_dicts/*')
to_classify = ["sample", "domain", "phylum", "class", "order", "family", "genus"]
classifier = "phylum"

In [27]:
# calculate thresholds

# find average value of each
avg_d = {}
total_organisms = {}
num_files = len(files)

for file in files:
    with open(file, 'rb') as f:
        taxa_dict = pickle.load(f)
    
    # count total organisms per patient
    organism_count = 0
    for i in taxa_dict[classifier]:
        organism_count += taxa_dict[classifier][i]
    total_organisms[file] = organism_count

    # add normalized data to the total, and divide by number of patients to average
    for i in taxa_dict[classifier]:
        if i not in avg_d:
            avg_d[i] = taxa_dict[classifier][i] / (num_files * organism_count)
        else:
            avg_d[i] += taxa_dict[classifier][i] / (num_files * organism_count)

In [28]:
# count number of patients both are above threshold in

total_files_per_sample = {}

for file in files:
    with open(file, 'rb') as f:
        taxa_dict = pickle.load(f)
        
    for i in taxa_dict[classifier]:
        if i not in total_files_per_sample:
                total_files_per_sample[i] = {}

        if taxa_dict[classifier][i] / total_organisms[file] < avg_d[i]:
            continue

        else:
            for j in taxa_dict[classifier]:
                if taxa_dict[classifier][j] / total_organisms[file] >= avg_d[j]:
                    if j not in total_files_per_sample[i]:
                        total_files_per_sample[i][j] = 1
                    else:
                        total_files_per_sample[i][j] += 1


In [29]:
max = 0

for i in total_files_per_sample:
    for j in total_files_per_sample[i]:
        if total_files_per_sample[i][j] > max:
            max = total_files_per_sample[i][j]

for i in total_files_per_sample:
    for j in total_files_per_sample[i]:
        total_files_per_sample[i][j] = 100 * total_files_per_sample[i][j] / max

In [30]:
df = pd.DataFrame.from_dict(total_files_per_sample, orient='index')

plt.figure(figsize=(15, 15), dpi=600)
coloring = sns.color_palette("ch:s=-.2,r=.6", as_cmap=True)
coloring.set_bad("#E3D8C1")
heatmap = sns.heatmap(df, annot = False, cmap = coloring, cbar_kws={'label': 'patients that both are upregulated in, normalized'})
heatmap.set_title("Correlation between taxa in " + disease.upper() + " ("+classifier+"), total patients = " + str(num_files))
# heatmap.tick_params(bottom = False)

Text(0.5, 1.0, 'Correlation between taxa in CRC (phylum), total patients = 42')